# Preprocessing arbitrarily structured data for AI with Awkward Array

**Vangelis Kourlitis, Technical University of Munich**  
*vangelis.kourlitis@cern.ch*  

Jim Pivarsky, Princeton University  
*pivarski@princeton.edu*

## Introduction

Usually the arrays you deal with are rectangular (in $n$ dimensions; "rectilinear").

<center>
<img src="img/8-layer_cube.jpg" width="50%">
</center>

What if we had data like this?

```json
[
  [[1.84, 0.324]],
  [[-1.609, -0.713, 0.005], [0.953, -0.993, 0.011, 0.718]],
  [[0.459, -1.517, 1.545], [0.33, 0.292]],
  [[-0.376, -1.46, -0.206], [0.65, 1.278]],
  [[], [], [1.617]],
  []
]
[
  [[-0.106, 0.611]],
  [[0.118, -1.788, 0.794, 0.658], [-0.105]]
]
[
  [[-0.384], [0.697, -0.856]],
  [[0.778, 0.023, -1.455, -2.289], [-0.67], [1.153, -1.669, 0.305, 1.517, -0.292]]
]
[
  [[0.205, -0.355], [-0.265], [1.042]],
  [[-0.004], [-1.167, -0.054, 0.726, 0.213]],
  [[1.741, -0.199, 0.827]]
]
```

Or **heterogenous** data like this?

```json
[
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 5.27453, "y": 1.03276},
    {"x": -3.51280, "y": 1.74849}]},
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 8.21630, "y": 4.07844},
    {"x": -0.79157, "y": 3.49478}, {"x": 16.38932, "y": 5.29399},
    {"x": 10.38641, "y": 0.10832}, {"x": -2.07070, "y": 14.07140},
    {"x": 9.57021, "y": -0.94823}, {"x": 1.97332, "y": 3.62380},
    {"x": 5.66760, "y": 11.38001}, {"x": 0.25497, "y": 3.39276},
    {"x": 3.86585, "y": 6.22051}, {"x": -0.67393, "y": 2.20572}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": 3.59528, "y": 7.37191},
    {"x": 0.59192, "y": 2.91503}, {"x": 4.02932, "y": -1.13601},
    {"x": -1.01593, "y": 1.95894}, {"x": 1.03666, "y": 0.05251}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": -8.78510, "y": -0.00497},
    {"x": -15.22688, "y": 3.90244}, {"x": 5.74593, "y": 4.12718}]},
  {"fill": "none", "stroke": "#000000", "points": [{"x": 4.40625, "y": -6.953125},
    {"x": 4.34375, "y": -7.09375}, {"x": 4.3125, "y": -7.140625},
    {"x": 4.140625, "y": -7.140625}]},
  {"fill": "none", "stroke": "#808080", "points": [{"x": 0.46875, "y": -0.09375},
    {"x": 0.46875, "y": -0.078125}, {"x": 0.46875, "y": 0.53125}]}
]
```

<br>

Real scientific datasets feature complex, irregular structures due to nested or variable-sized outputs from different sensors, or due to missing data values. The data can be also of mixed types or **heterogeneous**. Thus some level of data cleaning is almost always required.

**Goal**: process arbitrary data structure with array-oriented interface and performance...

<center>
<img src="img/awkward-motivation-venn-diagram.svg" width="40%">
</center>

### Libraries for irregular arrays

<table>
<tr style="background: white;"><td width="35%"><img src="img/logo-arrow.svg" width="100%"></td><td style="padding-left: 50px;">In-memory format and an ecosystem of tools, an "exploded database" (database functionality provided as interchangeable pieces). Strong focus on delivering data, zero-copy, between processes.</td></tr>
<tr style="background: white; height: 30px;"><td></td><td></td></tr>
<tr style="background: white;"><td width="35%"><img src="img/logo-awkward.svg" width="100%"></td><td style="padding-left: 50px;">Library for array-oriented programming like NumPy, but for arbitrary data structures. Losslessly zero-copy convertible to/from Arrow and Parquet.</td></tr>
<tr style="background: white; height: 30px;"><td></td><td></td></tr>
<tr style="background: white;"><td width="35%"><img src="img/logo-parquet.svg" width="100%"></td><td style="padding-left: 50px;">Disk format for storing large datasets and (selectively) retrieving them.</td></tr>
</table>

<img src="img/logo-arrow.svg" width="30%">

<br>

In [1]:
import pyarrow as pa

<br>

In [2]:
arrow_array = pa.array([
    [{"x": 1.1, "y": [1]}, {"x": 2.2, "y": [1, 2]}, {"x": 3.3, "y": [1, 2, 3]}],
    [],
    [{"x": 4.4, "y": [1, 2, 3, 4]}, {"x": 5.5, "y": [1, 2, 3, 4, 5]}]
])

<br>

In [3]:
arrow_array.type

ListType(list<item: struct<x: double, y: list<item: int64>>>)

<br>

<img src="img/logo-awkward.svg" width="30%">

<br>

In [4]:
import awkward as ak

<br>

In [5]:
awkward_array = ak.from_arrow(arrow_array)
awkward_array

<Array [[{x: 1.1, y: [1]}, ..., {...}], ...] type='3 * var * ?{x: ?float64,...'>

<img src="img/logo-parquet.svg" width="30%">

<br>

In [6]:
ak.to_parquet(awkward_array, "/tmp/file.parquet")

  created_by: parquet-cpp-arrow version 11.0.0
  num_columns: 2
  num_rows: 3
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

<br>

In [7]:
ak.from_parquet("/tmp/file.parquet")

<Array [[{x: 1.1, y: [1]}, ..., {...}], ...] type='3 * var * ?{x: ?float64,...'>

## Awkward Array

In [8]:
ragged = ak.Array([
    [
      [[1.84, 0.324]],
      [[-1.609, -0.713, 0.005], [0.953, -0.993, 0.011, 0.718]],
      [[0.459, -1.517, 1.545], [0.33, 0.292]],
      [[-0.376, -1.46, -0.206], [0.65, 1.278]],
      [[], [], [1.617]],
      []
    ],
    [
      [[-0.106, 0.611]],
      [[0.118, -1.788, 0.794, 0.658], [-0.105]]
    ],
    [
      [[-0.384], [0.697, -0.856]],
      [[0.778, 0.023, -1.455, -2.289], [-0.67], [1.153, -1.669, 0.305, 1.517, -0.292]]
    ],
    [
      [[0.205, -0.355], [-0.265], [1.042]],
      [[-0.004], [-1.167, -0.054, 0.726, 0.213]],
      [[1.741, -0.199, 0.827]]
    ]
])

**Multidimensional indexing**

In [9]:
ragged[3, 1, -1, 2]

0.726

<br>

**Basic slicing**

In [10]:
ragged[3, 1:, -1, 1:3]

<Array [[-0.054, 0.726], [-0.199, 0.827]] type='2 * var * float64'>

**Awkward slicing**

In [11]:
ragged > 0

<Array [[[[True, True]], [...], ..., []], ...] type='4 * var * var * var * ...'>

In [12]:
ragged[ragged > 0]

<Array [[[[1.84, 0.324]], [...], ..., []], ...] type='4 * var * var * var *...'>

**Reductions**

In [13]:
ak.sum(ragged)

2.8980000000000006

In [14]:
ak.sum(ragged, axis=-1)

<Array [[[2.16], [-2.32, ...], ..., []], ...] type='4 * var * var * float64'>

In [15]:
ak.sum(ragged, axis=0)

<Array [[[1.56, 0.58], [...], [1.04]], ..., []] type='6 * var * var * float64'>

How are reductions even defined for ragged arrays?

<center>
<img src="img/example-reduction-sum.svg" width="50%">
</center>

Let's demonstrate it with a small example:

In [16]:
small_ragged = ak.Array([[   1, 2, 4],
                         [          ],
                         [None, 8,  ],
                         [  16      ]])

In [17]:
ak.sum(small_ragged, axis=0)

<Array [17, 10, 4] type='3 * int64'>

### Heterogenous data

In [18]:
structured = ak.Array([
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 5.27453, "y": 1.03276},
    {"x": -3.51280, "y": 1.74849}]},
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 8.21630, "y": 4.07844},
    {"x": -0.79157, "y": 3.49478}, {"x": 16.38932, "y": 5.29399},
    {"x": 10.38641, "y": 0.10832}, {"x": -2.07070, "y": 14.07140},
    {"x": 9.57021, "y": -0.94823}, {"x": 1.97332, "y": 3.62380},
    {"x": 5.66760, "y": 11.38001}, {"x": 0.25497, "y": 3.39276},
    {"x": 3.86585, "y": 6.22051}, {"x": -0.67393, "y": 2.20572}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": 3.59528, "y": 7.37191},
    {"x": 0.59192, "y": 2.91503}, {"x": 4.02932, "y": -1.13601},
    {"x": -1.01593, "y": 1.95894}, {"x": 1.03666, "y": 0.05251}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": -8.78510, "y": -0.00497},
    {"x": -15.22688, "y": 3.90244}, {"x": 5.74593, "y": 4.12718}]},
  {"fill": "none", "stroke": "#000000", "points": [{"x": 4.40625, "y": -6.953125},
    {"x": 4.34375, "y": -7.09375}, {"x": 4.3125, "y": -7.140625},
    {"x": 4.140625, "y": -7.140625}]},
  {"fill": "none", "stroke": "#808080", "points": [{"x": 0.46875, "y": -0.09375},
    {"x": 0.46875, "y": -0.078125}, {"x": 0.46875, "y": 0.53125}]}
])

A lot of times you can use Awkward and Numpy interchangeably.

In [19]:
import numpy as np

**Elementwise formulas**

In [20]:
np.sqrt(structured["points", "x"]**2 + structured["points", "y"]**2)

<Array [[5.37, 3.92], ..., [0.478, 0.475, 0.708]] type='6 * var * float64'>

Alternative (and maybe more convenient) formulation:

In [21]:
np.sqrt(structured.points.x**2 + structured.points.y**2)

<Array [[5.37, 3.92], ..., [0.478, 0.475, 0.708]] type='6 * var * float64'>

Let's do an example calculation on those heterogenous data. Each row stores a `fill` color and a set of `points` in `x` and `y` coordinates, among other information. Consecutive `points` define _segments_ and the segments define a total _path_. We want to calculate the `fill` color of the path with the largest length.

We will calculate the length of the path in each row by summing the segment lengths $\displaystyle \sum_i^{n - 1} \Delta r_i$, where the segment length $\Delta r_i = \sqrt{\Delta x_i^2 + \Delta y_i^2}$

<br>

<center>
<img src="img/length-by-segment.svg" width="50%">
</center>

In [22]:
paths_length = np.sum(
    np.sqrt(
        (structured.points.x[:, 1:] - structured.points.x[:, 0:-1])**2 + (structured.points.y[:, 1:] - structured.points.y[:, 0:-1])**2
        )
    , axis=-1)
display(paths_length)

<Array [8.82, 110, 19.4, 28.5, 0.382, 0.625] type='6 * float64'>

In [23]:
color = structured[ np.argmax( paths_length ) ].fill

from IPython.display import Markdown, display
display(Markdown(
    f'<span style="font-family: monospace">{color} <span style="color: {color}">████████</span></span>'
))

<span style="font-family: monospace">#b1b1b1 <span style="color: #b1b1b1">████████</span></span>

### GPU capabilities

CUDA GPU kernels have been developed for a lot of Awkward operations (more currently under developments) thus Awkward array data can be to the device and the number crunching happens accelerated. 

In [24]:
# the exmple ragged array from above
array_cpu = ragged
ak.backend(array_cpu)

'cpu'

In [25]:
# move to the device
array_gpu = ak.to_backend(array_cpu, "cuda")
ak.backend(array_gpu)

'cuda'

In [26]:
# repeat the same calculation as above but on the divice
structured_gpu = ak.to_backend(structured, "cuda")
array_sum = np.sum(np.sqrt((structured_gpu.points.x[:, 1:] - structured_gpu.points.x[:, 0:-1])**2 + (structured_gpu.points.y[:, 1:] - structured_gpu.points.y[:, 0:-1])**2), axis=-1)

# display
display(array_sum[array_sum > 10]) # selection/indexing
display(array_sum[array_sum < 10]) # selection/indexing

# everything stays in the device
ak.backend(array_sum)

<Array [109.70188358249592, ..., 28.508228222215624] type='3 * float64'>

<Array [8.816430000958439, 0.382100141832189, 0.625] type='3 * float64'>

'cuda'

### Bonus: Combinatorics

Some operations are more meaningful on irregular arrays than rectilinear ones:

<table style="width: 60%">
<tr style="background: white; padding-top: 0px;"><td width="50%"><img src="img/cartoon-cartesian.svg" width="100%"></td><td width="50%"><img src="img/cartoon-combinations.svg" width="100%"></td></tr>
</table>

[ak.cartesian](https://awkward-array.org/doc/main/reference/generated/ak.cartesian.html) takes a [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of lists from $n$ different arrays, making an array of lists of $n$-tuples.

[ak.combinations](https://awkward-array.org/doc/main/reference/generated/ak.combinations.html) takes $n$ [samples without replacement](http://prob140.org/sp18/textbook/notebooks-md/5_04_Sampling_Without_Replacement.html) of lists from a single array, making an array of lists of $n$-tuples.

<center>
<img src="img/cartoon-cartesian.svg" width="30%">
</center>

In [27]:
numbers = ak.Array([[1, 2, 3], [], [4]])
letters = ak.Array([["a", "b"], ["c"], ["d", "e"]])

<br>

In [28]:
ak.cartesian([numbers, letters])

<Array [[(1, 'a'), (...), ..., (3, 'b')], ...] type='3 * var * (int64, string)'>

<center>
<img src="img/cartoon-combinations.svg" width="30%">
</center>

In [29]:
values = ak.Array([[1.1, 2.2, 3.3, 4.4], [], [5.5, 6.6]])

<br>

In [30]:
ak.combinations(values, 2)

<Array [[(1.1, 2.2), ..., (3.3, ...)], ...] type='3 * var * (float64, float64)'>

## Awkward as a preprocessor

Awkward Array helps transform data from its raw source into a format that ML models can use. Whether that involves padding data with zeros and applying masks for fixed-dimension inputs or providing data directly to models that support truly ragged arrays, Awkward Array makes this process easier.

Let's demonstrate that using some real HEP (ragged) data from the ATLAS experiment! **ATLAS has recently released 65 TB of PHYSLITE open data for research -- this is over 7 billion LHC collision events!** Those are all the data collected by the experiment during the 2015 and 2016. The release is accompanied by additional 2 billion events of simulated “Monte Carlo” data, which are essential for carrying out a physics analysis. The simulated data have largely the same structure as the real data. We're going to use these simulated events for today's demonstration for practical purposes.

Read about our open data release at: 

https://atlas.cern/Updates/News/Open-Data-Research

Our open data portal provides in depth information about the data along with analysis tutorials:

https://opendata.atlas.cern/

The properties of the particles we store in those files are all listed in:

https://atlas-physlite-content-opendata.web.cern.ch/

In [31]:
import uproot # the HEP-specific library to read the ATLAS data

You can download some example data (DOI:[10.7483/OPENDATA.ATLAS.K5SU.X65Y](http://doi.org/10.7483/OPENDATA.ATLAS.K5SU.X65Y)) with:  
`wget https://opendata.cern.ch/record/80010/files/assets/atlas/rucio/mc20_13TeV/DAOD_PHYSLITE.37621317._000001.pool.root.1`

In [32]:
example_file = '/afs/ipp-garching.mpg.de/home/e/eveko/ptmp/samples/DAOD_PHYSLITE.37621317._000001.pool.root.1'

We will open these data with `uproot` and load to Awkward arrays. We will use `ak.zip` to zip the two arrays in a single structure.

In [33]:
with uproot.open({example_file: 'CollectionTree'}) as tree:
    electrons = ak.zip({
        "pt": tree["AnalysisElectronsAuxDyn.pt"].array(), # continuous/float data 
        "id": tree["AnalysisElectronsAuxDyn.DFCommonElectronsLHTight"].array() # categorical data 
        })

In [34]:
electrons

<Array [[], ..., [{pt: 1.98e+05, ...}]] type='50000 * var * {pt: float32, i...'>

### Padding & Masking

Often ML frameworks and algorithms require rectilinear data structures. Awkward can help on cleaning and selecting ragged data and then deliver rectilinear arrays that can be converted to tensors. We will demonstrate how to pad and extract masks from rectilinear data. 

In [35]:
# select events (rows) with at least one electron 
selected_electrons = electrons[ ak.num( electrons ) > 0 ]
# and pt > 20000 MeV
selected_electrons = selected_electrons[ selected_electrons.pt > 20000 ]
# and "tight" identification quality
selected_electrons = selected_electrons[ selected_electrons.id == 1]
display(selected_electrons)

<Array [[{pt: 3.56e+04, id: 1}], ..., []] type='42435 * var * {pt: float32,...'>

Now let's assume we only care for the electrons energy (`pt`) for the downstream task.

In [36]:
selected_el_pt = selected_electrons.pt
display(selected_el_pt)

<Array [[3.56e+04], [1.01e+05, ...], ..., [], []] type='42435 * var * float32'>

Let's first pad each row of the data with `None` values. We need to set the maximum length of the row. We will estimate that from the data themselves (`max_num_el`). The result will be a rectilinear array.

In [37]:
# what is the maximum number of electrons (columns) in each event (rows)?
max_num_el = np.max(ak.num(selected_el_pt, axis=1))
padded_el_pt = ak.pad_none(selected_el_pt, max_num_el)
display(padded_el_pt)

<Array [[3.56e+04, None, None], ..., [None, ...]] type='42435 * var * ?float32'>

Before we substitute `None` with any value, we can extract a mask that might be helpful.

In [38]:
mask = ak.is_none(padded_el_pt, axis=1)
display(mask)

<Array [[False, True, True], ..., [True, ..., True]] type='42435 * var * bool'>

Finally, we can substitute `None` with a default value, e.g. 0.

In [39]:
padded_el_pt = ak.fill_none(padded_el_pt, 0)
display(padded_el_pt)

<Array [[3.56e+04, 0, 0], [...], ..., [0, 0, 0]] type='42435 * var * float64'>

Let's import `torch` and convert the above rectilinear array to a tensor in the GPU.

In [40]:
import torch

In [41]:
torch.tensor( padded_el_pt, dtype=torch.float32, device='cuda' )

tensor([[ 35576.5000,      0.0000,      0.0000],
        [100533.3906,  24967.7129,      0.0000],
        [ 36397.3750,      0.0000,      0.0000],
        ...,
        [ 64642.6719,  28024.9629,      0.0000],
        [     0.0000,      0.0000,      0.0000],
        [     0.0000,      0.0000,      0.0000]], device='cuda:0')

Unfortunately, this **currently** fails:   
`torch.tensor( selected_el_pt, dtype=torch.float32, device='cuda' )`

In [42]:
try:
    torch.tensor( selected_el_pt, dtype=torch.float32, device='cuda' )
except Exception as e:
    print(f"Failed with error: {e}")

Failed with error: expected sequence of length 1 at dim 1 (got 2)


### Bonus: Incorporating into PyTorch Dataset

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader
import vector
vector.register_awkward()

In [44]:
class RootDataset(Dataset):
    def __init__(self, root_file):
        super().__init__()
        self.root_file = root_file
        self.event_data = None
        self.preload_data()

    def preload_data(self):
        with uproot.open({self.root_file: 'CollectionTree'}) as tree:
            jets = ak.zip({
                "pt": tree["AnalysisJetsAuxDyn.pt"].array(), # float
                "eta": tree["AnalysisJetsAuxDyn.eta"].array(), # float
                "phi": tree["AnalysisJetsAuxDyn.phi"].array(), # float
                "mass": tree["AnalysisJetsAuxDyn.m"].array() # float
            }, with_name='Momentum4D')
            jets["QGTagger_NTracks"] = tree["AnalysisJetsAuxDyn.DFCommonJets_QGTagger_NTracks"].array() # int
        
        # select events with at least two jets
        self.event_data = jets[ak.num(jets, axis=-1) >= 2]

    def __len__(self):
        return len(self.event_data)

    def __getitem__(self, idx):
        jets = self.event_data[idx]
        
        # use the pair of jets with minimum delta R
        candidates = ak.combinations(jets, 2, axis=-1)
        jets1, jets2 = ak.unzip(candidates)
        delta_r = jets1.deltaR(jets2)
        candidates = candidates[ ak.argmin(delta_r, keepdims=True) ] 
        
        # ugly convolved line but it doesn't matter
        candidates_list = list( map(lambda x: list(x.values()), ak.to_list(candidates[0])) )
        features_tensor = torch.flatten( torch.tensor( candidates_list, dtype=torch.float32 ) )
        
        return features_tensor

In [45]:
dataset = RootDataset(example_file)
dataloader = DataLoader(dataset, batch_size=16)

In [46]:
# Iterate through the DataLoader
for batch in dataloader:
    # move to the device
    batch = batch.cuda()
    # show
    display(batch)
    break

tensor([[ 2.5912e+04,  3.7325e+00, -2.3966e+00,  0.0000e+00,  0.0000e+00,
          2.1824e+04,  3.6222e+00, -3.1338e+00,  5.5243e-03,  0.0000e+00],
        [ 1.1231e+05,  2.7021e+00,  3.0416e+00,  9.0469e+03,  1.0000e+00,
          2.6598e+04,  3.1597e+00, -2.4458e+00, -6.6612e-03,  0.0000e+00],
        [ 1.7412e+04,  7.1859e-01,  2.2374e+00,  3.9400e+03,  4.0000e+00,
          1.0879e+04,  8.9316e-01,  1.7958e+00,  1.3523e+03,  2.0000e+00],
        [ 4.1110e+04, -1.3567e+00, -1.9223e+00,  4.5168e+03,  1.0000e+00,
          1.1191e+04, -3.4316e-01, -2.6607e+00,  1.6033e+03,  0.0000e+00],
        [ 1.5532e+05, -2.1477e+00,  5.8242e-02,  6.5135e+03,  3.0000e+00,
          1.4314e+05, -1.6720e+00, -2.6067e-01,  1.1581e+04,  7.0000e+00],
        [ 3.0910e+04,  2.3689e+00, -2.7179e+00,  1.8939e+03,  3.0000e+00,
          1.2631e+04,  1.9590e+00,  2.6578e+00,  1.8918e+03,  0.0000e+00],
        [ 5.5512e+04,  2.5274e+00, -7.7819e-01,  5.5383e+03,  3.0000e+00,
          4.8310e+04,  2.8099e+0

### Graphs in PyTorch Geometric

We build a toy example of how data can be manipulated to be used as input in PyTorch Geometric.

Let's assume we have a nested, ragged dataset, where each line represents a particle track. Each track is composed by `[x, y]` coordinates. Particles can travel short or longer distances, thus tracks can have variable length, thus variable `[x, y]` measurements.

In [47]:
# [x, y] coordinates of the tracks
tracks = ak.Array(
    [
        [[5.274530,	1.032760], [-3.512800,	1.748490]], # track 1
        [[5.274530,	1.032760], [-3.512800,	1.748490], [-3.512800,	1.748490]], # track 2
        [[8.216300, 4.078440], [-0.791570, 3.494780], [16.389320, 5.293990], [10.386410, 0.108320], [-2.070700, 14.071400], [9.570210, -0.948230], [1.973320, 3.623800], [5.667600, 11.380010], [0.254970, 3.392760], [3.865850, 6.220510], [-0.673930, 2.205720]], # track 3
        [[3.595280, 7.371910], [0.591920, 2.915030], [4.029320, -1.136010], [-1.015930, 1.958940], [1.036660, 0.052510]], # track 4
        [[-8.785100, -0.004970], [-15.226880, 3.902440], [5.745930, 4.127180]], # track 5
        [[4.406250, -6.953125], [4.343750, -7.093750], [4.312500, -7.140625], [4.140625, -7.140625]], # track 6
        [[0.468750, -0.093750], [0.468750, -0.078125], [0.468750, 0.531250]] # track 7
    ]
)

# visualise to a ragged pd DataFrame ;)
ak.to_dataframe(tracks)

values
entry subentry subsubentry          
0     0        0            5.274530
               1            1.032760
      1        0           -3.512800
               1            1.748490
1     0        0            5.274530
...                              ...
6     0        1           -0.093750
      1        0            0.468750
               1           -0.078125
      2        0            0.468750
               1            0.531250

[62 rows x 1 columns]

We want to break this "track" relationship and treat the data just as multiple `[x, y]` coordinates -- just as a tracker detector signatures! We will use those cooridates to construct a Graph object using [`torch_geometric.data.Data`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html) class.

In [48]:
from torch_geometric.data import Data

In [49]:
coordinates = ak.flatten(tracks)
display(coordinates)

<Array [[5.27, 1.03], [...], ..., [0.469, 0.531]] type='31 * var * float64'>

The above coordinates will be our node _features_. We will use Awkward to build the _edges_ of a fully connected Graph. 

<center>
<img src="img/graph.png" width="50%">
</center>

This is currenly implemented in the custom function `get_edges` but ongoing developments in Awkward will allow such calculation natively.

In [50]:
def get_edges(nodes: ak.Array) -> torch.tensor:
    '''
    Calculate the edges of a fully connected graph.
    Return tensor that can be directly used in torch_geometric.data.Data
    '''
    number_of_nodes = ak.num(nodes, axis=0)
    counts = np.arange(0, number_of_nodes)
    reversed_counts = counts[::-1]
    
    # calculate all possible edges in both directions using ak.combinations
    combinations = ak.combinations(counts, 2, axis=0)
    reversed_combinations = ak.combinations(reversed_counts, 2, axis=0)[::-1]
    
    # zip the two set of edges to a single Akward Array
    # and convert it to a tensor
    edge_index_tensor = torch.tensor( ak.to_list( ak.zip( [combinations, reversed_combinations] ) ), dtype=torch.long )
    # get the right shape
    edge_index_tensor = torch.flatten(edge_index_tensor).reshape(-1,2)
    
    return edge_index_tensor

Finally, we construct a `Data` object from `features_tensor` and an `edge_index_tensor`.

In [51]:
features_tensor = torch.tensor(ak.to_list(coordinates), dtype=torch.float)
edge_index_tensor = get_edges(coordinates)

data = Data(x=features_tensor, edge_index=edge_index_tensor.t().contiguous())

In [52]:
data.num_edges

930

## Awknowledgments

We thank [IRIS-HEP](https://iris-hep.org/) for providing the financial resources to conduct this demonstration and the [ORIGINS Data Science Laboratory](https://www.origins-cluster.de/en/infrastructure/odsl) for providing the required compute infrastructure.